https://neptune.ai/blog/web-scraping-and-knowledge-graphs-machine-learning

In [1]:

import wikipediaapi  
import pandas as pd
import concurrent.futures
from tqdm import tqdm


In [4]:
def scrape_wikipedia(name_topic, verbose=True):
   def link_to_wikipedia(link):
       try:
           page = api_wikipedia.page(link)
           if page.exists():
               return {'page': link, 'text': page.text, 'link': page.fullurl, 'categories': list(page.categories.keys())}
       except:
           return None
      
   api_wikipedia = wikipediaapi.Wikipedia(language='en',user_agent="Mozilla/5.0 (X11; Linux i686; rv:109.0) Gecko/20100101 Firefox/121.0", extract_format=wikipediaapi.ExtractFormat.WIKI)
   name_of_page = api_wikipedia.page(name_topic)
   if not name_of_page.exists():
       print('Page {} is not present'.format(name_of_page))
       return
  
   links_to_page = list(name_of_page.links.keys())
   procceed = tqdm(desc='Scraped links', unit='', total=len(links_to_page)) if verbose else None
   origin = [{'page': name_topic, 'text': name_of_page.text, 'link': name_of_page.fullurl, 'categories': list(name_of_page.categories.keys())}]
  
   with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
       links_future = {executor.submit(link_to_wikipedia, link): link for link in links_to_page}
       for future in concurrent.futures.as_completed(links_future):
           info = future.result()
           origin.append(info) if info else None
           procceed.update(1) if verbose else None
   procceed.close() if verbose else None
  
   namespaces = ('Wikipedia', 'Special', 'Talk', 'LyricWiki', 'File', 'MediaWiki',
                 'Template', 'Help', 'User', 'Category talk', 'Portal talk')
   origin = pds.DataFrame(origin)
   origin = origin[(len(origin['text']) > 20)
                     & ~(origin['page'].str.startswith(namespaces, na=True))]
   origin['categories'] = origin.categories.apply(lambda a: [b[9:] for b in a])

   origin['topic'] = name_topic
   print('Scraped pages', len(origin))
  
   return origin


In [6]:
data_wikipedia = scrape_wikipedia('Artificial_intelligence')

TypeError: Wikipedia.__init__() missing 1 required positional argument: 'user_agent'

In [ ]:
data_wikipedia.to_csv('scraped_data.csv')

Now that we have the data stored as csv we want to create the graph

In [ ]:
# download spacy model
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
import pandas as pd
import requests
from spacy import displacy
# import en_core_web_sm
 
nlp = spacy.load('en_core_web_sm')
 
from spacy.tokens import Span
from spacy.matcher import Matcher
 
import matplotlib.pyplot as plot
from tqdm import tqdm
import networkx as ntx
import neptune
 
%matplotlib inline


In [ ]:
run = neptune.init_run(api_token="your API key",
                   project="aravindcr/KnowledgeGraphs")

In [ ]:
# Upload data to Neptune.
run["data"].upload("scraped_data.csv")

In [ ]:

data = pd.read_csv('scraped_data.csv')